<a href="https://colab.research.google.com/github/shidan619/FYP/blob/main/Module%202.3-Aspect_auto_categorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Libraries

In [1]:
!pip install transformers
!pip install --upgrade keras
import pandas as pd
import numpy as np
import tensorflow as tf
import torch
import keras.preprocessing.sequence
from torch.nn import BCEWithLogitsLoss, BCELoss
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix, f1_score, accuracy_score
import pickle
from transformers import BertForSequenceClassification,BertTokenizer,AdamW
from tqdm import tqdm, trange
from ast import literal_eval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 78.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 47.5 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires keras<2.13,>=2.12.0, but you have keras 2.13.1 which is incompatible.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla T4'

## Load and Preprocess Training Data

Dataset will be tokenized then split into training and validation sets. The validation set will be used to monitor training. For testing a separate test set will be loaded for analysis.

In [5]:
# Step 1: Prepare your dataset
df = pd.read_csv('/content/drive/MyDrive/zidan/aspect_extracted_file.csv')

In [6]:
# Assuming your dataset is stored in X and y variables
df, test_data = train_test_split(df, test_size=0.1)


In [7]:
df

,product_name,review,review_without_emoji,preprocess_text,preprocess_text1,aspects,price,quality,delivery,packaging,design,customer service,usability
9739,HINSOCHA 3 Pairs Rubber Cleaning Gloves Latex ...,These gloves fit well on smaller hands. The ru...,These gloves fit well on smaller hands. The ru...,these gloves fit well on smaller hands the rub...,"['gloves', 'fit', 'well', 'smaller', 'hands', ...","['quality', 'usability']",0,1,0,0,0,0,1
6079,Plant Based - Hippo Sak Tall Kitchen Bags with...,I have searched for suitable trash bags for my...,I have searched for suitable trash bags for my...,i have searched for suitable trash bags for my...,"['searched', 'suitable', 'trash', 'bags', 'tal...",['design'],0,0,0,0,1,0,0
5341,"Emcgicc Retractable Gap Dust Cleaner, Microfib...",I bought this because it has a telescoping han...,I bought this because it has a telescoping han...,i bought this because it has a telescoping han...,"['bought', 'telescoping', 'handle', 'great', '...","['quality', 'design', 'usability']",0,1,0,0,1,0,1
5597,Skura Style Kitchen Sponges – Odor Free Dual S...,This is about as close to a perfect sponge to ...,This is about as close to a perfect sponge to ...,this is about as close to a perfect sponge to ...,"['close', 'perfect', 'sponge', 'use', 'kitchen...","['price', 'quality', 'delivery', 'customer ser...",1,1,1,0,0,1,1
5663,Comfy Package [300 Count] 3 oz. White Paper Cu...,Holds mouth wash for 30 second rinse,Holds mouth wash for 30 second rinse,holds mouth wash for 30 second rinse,"['holds', 'mouth', 'wash', '30', 'second', 'ri...",['quality'],0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8892,"2 Gallon Small Trash Bags, Clear, 150 Counts/ ...",All 3 rate features,All 3 rate features,all 3 rate features,"['3', 'rate', 'features']",[],0,0,0,0,0,0,0
7303,"MR.SIGA Lint Free Microfiber Duster, Washable ...",I recently had shutter blinds installed in my ...,I recently had shutter blinds installed in my ...,i recently had shutter blinds installed in my ...,"['recently', 'shutter', 'blinds', 'installed',...","['price', 'design']",1,0,0,0,1,0,0
1724,"Reynolds Wrap Heavy Duty Aluminum Foil, 50 Squ...",My favorite foil! Very strong,My favorite foil! Very strong,my favorite foil very strong,"['favorite', 'foil', 'strong']",['quality'],0,1,0,0,0,0,0
889,"Brawny® Pick-A-Size® Paper Towels, 12 Double R...","I know, it’s just paper towels. What makes the...","I know, it’s just paper towels. What makes the...",i know its just paper towels what makes them s...,"['know', 'paper', 'towels', 'makes', 'special'...","['price', 'quality', 'delivery', 'usability']",1,1,1,0,0,0,1


In [8]:
columns_to_drop = ['product_name', 'preprocess_text','review_without_emoji', 'aspects', 'preprocess_text1']
df.drop(columns_to_drop, axis=1, inplace=True)

In [9]:
df.head()

,review,price,quality,delivery,packaging,design,customer service,usability
9739,These gloves fit well on smaller hands. The ru...,0,1,0,0,0,0,1
6079,I have searched for suitable trash bags for my...,0,0,0,0,1,0,0
5341,I bought this because it has a telescoping han...,0,1,0,0,1,0,1
5597,This is about as close to a perfect sponge to ...,1,1,1,0,0,1,1
5663,Holds mouth wash for 30 second rinse,0,1,0,0,0,0,0


In [10]:
print('Unique comments: ', df.review.nunique() == df.shape[0])
print('Null values: ', df.isnull().values.any())

Unique comments:  False
Null values:  False


In [11]:
print('average sentence length: ', df.review.str.split().str.len().mean())  #Average Sentence length
print('stdev sentence length: ', df.review.str.split().str.len().std())

average sentence length:  46.441563661943526
stdev sentence length:  85.65297623912458


In [12]:
cols = df.columns
label_cols = list(cols[1:])
num_labels = len(label_cols)
print('Label columns: ', label_cols)

Label columns:  ['price', 'quality', 'delivery', 'packaging', 'design', 'customer service', 'usability']


In [13]:
df = df.sample(frac=1).reset_index(drop=True) #shuffle rows

In [14]:
df['labels'] = list(df[label_cols].values)
df.head()

,review,price,quality,delivery,packaging,design,customer service,usability,labels
0,"Like a lot of folks, I have various devices th...",0,1,0,0,0,0,1,"[0, 1, 0, 0, 0, 0, 1]"
1,the product looks great and meets expectations.,0,0,0,0,1,0,0,"[0, 0, 0, 0, 1, 0, 0]"
2,Good bags to pick up poop You might need two t...,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0]"
3,Super product for the gloves,0,1,0,0,0,0,0,"[0, 1, 0, 0, 0, 0, 0]"
4,"It fits me good, but it is thin. I am not sure...",0,1,0,0,0,0,1,"[0, 1, 0, 0, 0, 0, 1]"


In [15]:
labels = list(df.labels.values)
comments = list(df.review.values)

In [16]:
max_length = 128
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True) # tokenizer
encodings = tokenizer.batch_encode_plus(comments,max_length=max_length,pad_to_max_length=True) # tokenizer's encoding method
print('tokenizer outputs: ', encodings.keys())

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


tokenizer outputs:  dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


In [17]:
input_ids = encodings['input_ids'] # tokenized and encoded sentences
token_type_ids = encodings['token_type_ids'] # token type ids
attention_masks = encodings['attention_mask'] # attention masks

In [18]:
# Identifying indices of 'one_hot_labels' entries that only occur once - this will allow us to stratify split our training data later
label_counts = df.labels.astype(str).value_counts()
one_freq = label_counts[label_counts==1].keys()
one_freq_idxs = sorted(list(df[df.labels.astype(str).isin(one_freq)].index), reverse=True)
print('df label indices with only one instance: ', one_freq_idxs)

df label indices with only one instance:  [9579, 8575, 5984, 5704, 5438, 3805, 3640, 2794, 2620, 968]


In [19]:
# Gathering single instance inputs to force into the training set after stratified split
one_freq_input_ids = [input_ids.pop(i) for i in one_freq_idxs]
one_freq_token_types = [token_type_ids.pop(i) for i in one_freq_idxs]
one_freq_attention_masks = [attention_masks.pop(i) for i in one_freq_idxs]
one_freq_labels = [labels.pop(i) for i in one_freq_idxs]

Be sure to handle all classes during validation using "stratify" during train/validation split:

In [20]:
# Use train_test_split to split our data into train and validation sets

train_inputs, validation_inputs, train_labels, validation_labels, train_token_types, validation_token_types, train_masks, validation_masks = train_test_split(input_ids, labels, token_type_ids,attention_masks,
                                                            random_state=2020, test_size=0.10, stratify = labels)

# Add one frequency data to train data
train_inputs.extend(one_freq_input_ids)
train_labels.extend(one_freq_labels)
train_masks.extend(one_freq_attention_masks)
train_token_types.extend(one_freq_token_types)

# Convert all of our data into torch tensors, the required datatype for our model
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
train_token_types = torch.tensor(train_token_types)

validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)
validation_token_types = torch.tensor(validation_token_types)

<ipython-input-20-8effe635eb5d>:14: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  train_labels = torch.tensor(train_labels)


In [21]:
# Select a batch size for training. We will use 32 here to avoid memory issues.
batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels, train_token_types)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels, validation_token_types)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [22]:
torch.save(validation_dataloader,'validation_data_loader')
torch.save(train_dataloader,'train_data_loader')

## Load Model & Set Params

In [23]:
# Load model, the pretrained model will include a single linear classification layer on top for classification.
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

Setting custom optimization parameters for the AdamW optimizer https://huggingface.co/transformers/main_classes/optimizer_schedules.html

In [24]:
# setting custom optimization parameters. You may implement a scheduler here as well.
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

In [27]:
optimizer = AdamW(optimizer_grouped_parameters,lr=5e-5,correct_bias=True)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


## Train Model

In [28]:
# Store our loss and accuracy for plotting
train_loss_set = []

# Number of training epochs
epochs = 3

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):

  # Training

  # Set our model to training mode (as opposed to evaluation mode)
  model.train()

  # Tracking variables
  tr_loss = 0 #running loss
  nb_tr_examples, nb_tr_steps = 0, 0

  # Train the data for one epoch
  for step, batch in enumerate(train_dataloader):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels, b_token_types = batch
    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()

    # Forward pass for multilabel classification
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    logits = outputs[0]
    loss_func = BCEWithLogitsLoss()
    loss = loss_func(logits.view(-1,num_labels),b_labels.type_as(logits).view(-1,num_labels)) #convert labels to float for calculation
    # loss_func = BCELoss()
    # loss = loss_func(torch.sigmoid(logits.view(-1,num_labels)),b_labels.type_as(logits).view(-1,num_labels)) #convert labels to float for calculation
    train_loss_set.append(loss.item())

    # Backward pass
    loss.backward()
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    # Update tracking variables
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

  print("Train loss: {}".format(tr_loss/nb_tr_steps))

###############################################################################

  # Validation

  # Put model in evaluation mode to evaluate loss on the validation set
  model.eval()

  # Variables to gather full output
  logit_preds,true_labels,pred_labels,tokenized_texts = [],[],[],[]

  # Predict
  for i, batch in enumerate(validation_dataloader):
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels, b_token_types = batch
    with torch.no_grad():
      # Forward pass
      outs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
      b_logit_pred = outs[0]
      pred_label = torch.sigmoid(b_logit_pred)

      b_logit_pred = b_logit_pred.detach().cpu().numpy()
      pred_label = pred_label.to('cpu').numpy()
      b_labels = b_labels.to('cpu').numpy()

    tokenized_texts.append(b_input_ids)
    logit_preds.append(b_logit_pred)
    true_labels.append(b_labels)
    pred_labels.append(pred_label)

  # Flatten outputs
  pred_labels = [item for sublist in pred_labels for item in sublist]
  true_labels = [item for sublist in true_labels for item in sublist]

  # Calculate Accuracy
  threshold = 0.50
  pred_bools = [pl>threshold for pl in pred_labels]
  true_bools = [tl==1 for tl in true_labels]
  val_f1_accuracy = f1_score(true_bools,pred_bools,average='micro')*100
  val_flat_accuracy = accuracy_score(true_bools, pred_bools)*100

  print('F1 Validation Accuracy: ', val_f1_accuracy)
  print('Flat Validation Accuracy: ', val_flat_accuracy)

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Train loss: 0.09624992708808609


Epoch:  33%|███▎      | 1/3 [03:05<06:10, 185.10s/it]

F1 Validation Accuracy:  96.54028436018957
Flat Validation Accuracy:  88.84422110552764
Train loss: 0.05955950471439532


Epoch:  67%|██████▋   | 2/3 [06:09<03:04, 184.56s/it]

F1 Validation Accuracy:  96.715157203191
Flat Validation Accuracy:  89.54773869346734
Train loss: 0.04119267305359244


Epoch: 100%|██████████| 3/3 [09:13<00:00, 184.50s/it]

F1 Validation Accuracy:  96.87866697958225
Flat Validation Accuracy:  90.65326633165829


In [29]:
import os

output_dir = os.path.expanduser('/content/drive/MyDrive/zidan/finetuned_bert')

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print('Saved')

Saved


## Load and Preprocess Test Data

In [30]:
test_data.head()

,product_name,review,review_without_emoji,preprocess_text,preprocess_text1,aspects,price,quality,delivery,packaging,design,customer service,usability
8220,HINSOCHA 3 Pairs Rubber Cleaning Gloves Latex ...,These are very nice rubber gloves. I like that...,These are very nice rubber gloves. I like that...,these are very nice rubber gloves i like that ...,"['nice', 'rubber', 'gloves', 'like', 'could', ...","['quality', 'packaging', 'usability']",0,1,0,1,0,0,1
4594,"OXO SteeL Soap Dispensing Palm Brush Refills,2...",I have a bunch of refills for the stainless st...,I have a bunch of refills for the stainless st...,i have a bunch of refills for the stainless st...,"['bunch', 'refills', 'stainless', 'steel', 'on...","['quality', 'delivery', 'packaging', 'design',...",0,1,1,1,1,0,1
4840,100% Compostable 9 Inch Heavy-Duty Paper Plate...,Summer staple. Want time away from dishes whe...,Summer staple. Want time away from dishes whe...,summer staple want time away from dishes when...,"['summer', 'staple', 'want', 'time', 'away', '...","['quality', 'delivery']",0,1,1,0,0,0,0
6583,"Pacific Blue Ultra 9"" Paper Towel Roll (Previo...",The fabric on these papers isn't as good as it...,The fabric on these papers isn't as good as it...,the fabric on these papers isnt as good as it ...,"['fabric', 'papers', 'isnt', 'good', 'used', '...","['price', 'quality', 'design', 'usability']",1,1,0,0,1,0,1
3778,"Fantastik All-Purpose Cleaner, With Bleach, 32...",Seems to work OK. Should have read more speci...,Seems to work OK. Should have read more speci...,seems to work ok should have read more specif...,"['seems', 'work', 'ok', 'read', 'specifics', '...",['quality'],0,1,0,0,0,0,0


In [31]:
columns_to_drop = ['product_name', 'aspects', 'review','review_without_emoji','preprocess_text1']
test_data.drop(columns_to_drop, axis=1, inplace=True)

test_label_cols = list(test_data.columns[1:])
print('Null values:', test_data.isnull().values.any())  # Should not be any null sentences or labels
print('Same columns between train and test:', label_cols == test_label_cols)  # Columns should be the same

test_data[test_label_cols] = -1  # Set label column values to -1

Null values: False
Same columns between train and test: True


In [32]:
test_data = test_data.replace(-1, 0)  # Change -1 to 0 in the entire DataFrame

test_data = test_data[~test_data[test_label_cols].eq(-1).any(axis=1)]  # Remove irrelevant rows/comments with -1 values
test_data['labels'] = list(test_data[test_label_cols].values)

In [33]:
# Gathering input data
test_labels = list(test_data.labels.values)
test_comments = list(test_data.preprocess_text.values)

In [34]:
# Encoding input data
test_encodings = tokenizer.batch_encode_plus(test_comments,max_length=max_length,pad_to_max_length=True)
test_input_ids = test_encodings['input_ids']
test_token_type_ids = test_encodings['token_type_ids']
test_attention_masks = test_encodings['attention_mask']

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [35]:
# Make tensors out of data
test_inputs = torch.tensor(test_input_ids)
test_labels = torch.tensor(test_labels)
test_masks = torch.tensor(test_attention_masks)
test_token_types = torch.tensor(test_token_type_ids)
# Create test dataloader
test_data = TensorDataset(test_inputs, test_masks, test_labels, test_token_types)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)
# Save test dataloader
torch.save(test_dataloader,'test_data_loader')

## Prediction and Metics

In [36]:
# Test

# Put model in evaluation mode to evaluate loss on the validation set
model.eval()

# Variables to gather full output
logit_preds,true_labels,pred_labels,tokenized_texts = [],[],[],[]

# Predict
for i, batch in enumerate(validation_dataloader):
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels, b_token_types = batch
  with torch.no_grad():
    # Forward pass
    outs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    b_logit_pred = outs[0]
    pred_label = torch.sigmoid(b_logit_pred)

    b_logit_pred = b_logit_pred.detach().cpu().numpy()
    pred_label = pred_label.to('cpu').numpy()
    b_labels = b_labels.to('cpu').numpy()

  tokenized_texts.append(b_input_ids)
  logit_preds.append(b_logit_pred)
  true_labels.append(b_labels)
  pred_labels.append(pred_label)

# Flatten outputs
tokenized_texts = [item for sublist in tokenized_texts for item in sublist]
pred_labels = [item for sublist in pred_labels for item in sublist]
true_labels = [item for sublist in true_labels for item in sublist]

# Calculate Accuracy
threshold = 0.50
pred_bools = [pl>threshold for pl in pred_labels]
true_bools = [tl==1 for tl in true_labels]
f1_accuracy = f1_score(true_bools,pred_bools,average='micro')*100
flat_accuracy = accuracy_score(true_bools, pred_bools)*100

print('Test F1 Accuracy: ', val_f1_accuracy)
print('Test Flat Accuracy: ', val_flat_accuracy)

clf_report = classification_report(true_bools,pred_bools,target_names=test_label_cols)
pickle.dump(clf_report, open('classification_report.txt','wb')) #save report

Test F1 Accuracy:  96.87866697958225
Test Flat Accuracy:  90.65326633165829


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Output Dataframe

In [37]:
idx2label = dict(zip(range(7),label_cols))
print(idx2label)

{0: 'price', 1: 'quality', 2: 'delivery', 3: 'packaging', 4: 'design', 5: 'customer service', 6: 'usability'}


In [38]:
# Getting indices of where boolean one hot vector true_bools is True so we can use idx2label to gather label names
true_label_idxs, pred_label_idxs=[],[]
for vals in true_bools:
  true_label_idxs.append(np.where(vals)[0].flatten().tolist())
for vals in pred_bools:
  pred_label_idxs.append(np.where(vals)[0].flatten().tolist())

In [39]:
# Gathering vectors of label names using idx2label
true_label_texts, pred_label_texts = [], []
for vals in true_label_idxs:
  if vals:
    true_label_texts.append([idx2label[val] for val in vals])
  else:
    true_label_texts.append(vals)

for vals in pred_label_idxs:
  if vals:
    pred_label_texts.append([idx2label[val] for val in vals])
  else:
    pred_label_texts.append(vals)

In [40]:
# Decoding input ids to reviews
reviews = [tokenizer.decode(text,skip_special_tokens=True,clean_up_tokenization_spaces=False) for text in tokenized_texts]

In [41]:
# Converting lists to df
comparisons_df = pd.DataFrame({'reviews': reviews,'pred_labels':pred_label_texts})
comparisons_df.to_csv('/content/drive/MyDrive/zidan/test_predictions.csv')
comparisons_df


,reviews,pred_labels
0,love it easy to use and it ’ s rechargeable . ...,"[quality, design, usability]"
1,i love these bags . they are thick and sturdy ...,"[price, quality, design]"
2,i didn ’ t even think that a mop could be so s...,"[price, quality, delivery, packaging, design, ..."
3,i got this to replace a lost stanley straw . i...,"[price, quality, packaging]"
4,"as anyone cant attest , prices on everything a...","[price, delivery, customer service]"
...,...,...
990,"very easy to put together , love the options f...","[quality, design]"
991,i love that you get this many and so quickly ....,"[quality, delivery, customer service]"
992,i really like the bomves electric spin scrubbe...,"[quality, design, usability]"
993,i don ’ t know why i never thought about purch...,"[quality, design]"
